In [1]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.2.22
    Uninstalling langgraph-0.2.22:
      Successfully uninstalled langgraph-0.2.22


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
local_llm = 'llama3.1'

LOAD BLOGPOSTS FROM INTERNET

RETRIEVER

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import  filter_complex_metadata
from langchain.docstore.document import Document

load_dotenv()

# Public urls to the blog post
urls = [
    "https://elsys-bg.org/priem/den-na-otvorenite-vrati",
    "https://tuesfest.bg/",
    "https://hacktues.bg/"
]


firecrawl_api_key = os.getenv('FIRECRAWL_API_KEY')

# Load the documents
docs = [FireCrawlLoader(api_key=firecrawl_api_key, url=url, mode="scrape").load() for url in urls]

# Flatten the list of lists
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=512, chunk_overlap=50)

doc_splits = text_splitter.split_documents(docs_list)

# Filter out complex metadata and ensure proper document format
filtered_docs = []
for doc in doc_splits:
    # Ensure the doc is instance of Document and has proper metadata
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_docs.append(Document(page_content=doc.page_content, metadata=clean_metadata))
        
# Add to vector DB
vector_store = Chroma.from_documents(
    documents=filtered_docs,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(),
)

retriever = vector_store.as_retriever()

/Users/flychuban/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


I will use Retrieval Grader for checking wheter the retrieved documents are fine

RETRIEVAL GRADER

In [13]:
from langchain.prompts import  PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

#LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a grader assessing relevance of a retrieved document to a user question. 
    If the document contains keywords related to the user question, grade it as a relevant. It does not need to bea stringent test. The goal is to filter out erroneous retrievals.\n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the user question. \n
    Provide the binary score as a JSON string with a single key 'score' and no premable or explanaiton.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieval document: \n \n {document} \n\n
    Here is the user question: \n \n {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>,
    """,
    input_variables=["questions", "document"],
)

# Define the grader using overloading pipeline
retrieval_grader = prompt | llm | JsonOutputParser()
user_question = "When was the first Hack TUES?"
docs = retriever.invoke(user_question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": user_question, "document": doc_txt}))

{'score': 'yes'}


GENERATE ANSWER

In [14]:
from langchain.prompts import  PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser


# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the questions.
    If you don't know the answer, respond with 'I don't know'. Use three sentences maximum and keep the answers concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>,
    """,
    input_variables=["questions", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Post processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain 
reg_chain = prompt | llm | StrOutputParser()

user_question = "When was the first Hack TUES?"
docs = retriever.invoke(user_question)
generation = reg_chain.invoke({"question": user_question, "context": format_docs(docs)})
print(generation)

The first Hack TUES was in 2019. It had a theme of "Polza na ednevna rabota na biznesa i zhivota na horata" (Daily work of business and people's life). The event took place from March 15-17, 2019.


HALLUCINATION GRADER

In [18]:
llm = ChatOllama(model=local_llm, temperature=0, format="json")

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assesing whether an answer is grounded in / supported by a set of facts. Give a binary score 
    'yes' or 'no' to indicate whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the facts:
    \n -------- \n 
    {documents}
    \n -------- \n
    Here is the answer: {generation} <|eot_id|><|start_header_id|>assistant<|end_header_id|>,
    """,
    input_variables=["questions", "document"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'true'}

ANSWER GRADER

In [25]:
llm = ChatOllama(model=local_llm, temperature=0, format="json")

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an answer is useful to resolve a question. Give a binary score 'yes' or 'no'
    to indicate whether the answer is useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n -------- \n
    {generation}
    \n -------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>,
    """,
    input_variables=["questions", "document"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"generation": generation, "question": user_question})

{'score': 'yes'}